In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as image
import os
import cv2

In [ ]:
capture = cv2.VideoCapture(0)

In [ ]:
#will hold the facial data
face_data = []
#will hold the class id identifying the face
face_id = []
class_id = 0
id_name_dict = {}
#browsing through saved faces in directory
for file in os.listdir('./data'):
    #loading a face into data_item
    data_item = np.load('./data/' + file)
    #appending into face data
    face_data.append(data_item)
    id_name_dict[str(class_id)] = str(file).split('.')[0]
    #each file is only one person but many faces, so we append same class_id for each face in that file
    face_id.append(class_id * np.ones((data_item.shape[0],)))
    #incrementing class_id
    class_id += 1
#converting to array
face_data = np.array(face_data)
face_id = np.array(face_id)
#concatenate, will give same results as flattening / resize, just adjusting shape here
f_face = np.concatenate(face_data, axis = 0)
f_fid = np.concatenate(face_id, axis = 0).reshape((-1,1))

f_face = f_face.reshape((f_face.shape[0],-1 ))
f_fid = f_fid.ravel()

print(f_face.shape, f_fid.shape)
print(id_name_dict)

In [ ]:
def dist(query, data):
    mydist = np.sqrt(((query - data)**2).sum())
    return mydist

def KNN(query, training_data, training_class, k):
    points = []
    for num in range(0,training_data.shape[0]):
        curdist = dist(query, training_data[num])
        points.append((curdist, training_class[num]))
    points = np.array(sorted(points))
    arr = points[:k,1]
    t = np.unique(arr, return_counts=True)
    idx = t[1].argmax()
    return t[0][idx]

In [ ]:
face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_alt.xml')
while True:
    #ret is a return code, frame is the frame captured
    ret, frame = capture.read()
    #convert to grayscale if needed
    grayscale = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #ret is false if frame was not captured properly, so we ignore that particular frame
    if ret == False:
        continue
    #stores the faces coordinates in rectangle start, width and height
    faces = face_cascade.detectMultiScale(grayscale, 1.3, 5)
    #iterating over each face detected in that particular frame
    for face in faces:
        #extracting our values from face
        x,y,w,h = face
        #offset value to increase our displayed rectangle size
        offset = 50
        #creating a rectangle using the coordinates known, top-left and bottom-right, color, and width
        cv2.rectangle(grayscale, (x - offset, y - offset), (x+w+offset, y+h+offset), (255,255,255), 4)
        #Slice of frame to select out face part
        face_section = grayscale[y - offset : y + h + offset, x - offset : x + w + offset]
        #resizing face section to a particular standard so that we will be able to apply KNN later
        face_section = cv2.resize(face_section, (100,100))
        flattened = face_section.flatten()
        #prediction
        mypredict = KNN(flattened, f_face, f_fid, 10)
        #making the rectangle and the text over it
        cv2.putText(grayscale, str(id_name_dict[str(int(mypredict))]), (x, y - 10),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0), 2, cv2.LINE_AA)
    cv2.imshow("Face Detection", grayscale)
    #first and operation basically gets key from user and compares it to binary equivalent of mentioned key
    #loop breaks if key matches
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
#releases capture interface
capture.release()
#closes any windows created, not doing so will cause process to freeze
cv2.destroyAllWindows()